In [1]:
import pandas as pd
import numpy as np
import pickle
import regex as re
from underthesea import word_tokenize

In [2]:
df_news = pd.read_csv('data_final_test.csv')

In [3]:
df_news.head()

,URL,Title,Summary,Contents,Date,Author(s),Category,Tags
0,https://laodong.vn/bat-dong-san/ha-noi-ho-bien...,Hà Nội: Hô biến “đất vàng” thành chung cư cao ...,Sau khi di dời các cơ sở công nghiệp ra khỏi n...,"Sau cổ phần hóa, “đất vàng” thành chung cư Khả...","Thứ bảy, 09/09/2017 10:30 (GMT+7)",THÔNG CHÍ,Bất động sản,['Hà Nội']
1,https://laodong.vn/bat-dong-san/ha-noi-va-tphc...,Hà Nội và TP.HCM đối mặt với suy giảm nguồn cu...,Nguồn cung căn hộ giảm trên tất cả các phân kh...,Tại Báo cáo Kinh tế vĩ mô Quý I 2019 công bố m...,"Thứ hai, 15/04/2019 17:00 (GMT+7)",Đ.Phương,Bất động sản,"['TPHCM', 'Hà Nội', 'TP.HCM', 'giá căn hộ chun..."
2,https://laodong.vn/bat-dong-san/an-theo-tp-thu...,"""Ăn theo"" Tp. Thủ Đức và sân bay Long Thành: S...",Cơn sốt đất chạy theo thông tin thành lập Thàn...,Giá đất tăng chóng mặt Có thể nói khắp các co...,"Thứ hai, 11/01/2021 09:45 (GMT+7)",Bảo Chương - Hữu Huy,Bất động sản,"['Sân bay Long Thành', 'Nhà đầu tư', 'Sốt đất'..."
3,https://laodong.vn/bat-dong-san/mua-nha-o-hinh...,Mua nhà ở hình thành trong tương lai và những ...,Những kiến thức cơ bản về mua nhà ở hình thành...,Nhà ở hình thành trong tương lai là gì? Nhà ở ...,"Thứ ba, 12/01/2021 18:36 (GMT+7)",Trang Thiều (T/H),Bất động sản,"['nhà ở', 'mua nhà ở', 'Nhà ở tương lai']"
4,https://laodong.vn/bat-dong-san/condotel-e-am-...,"Condotel ""ế ẩm"", nhà đầu tư khủng hoảng",TPHCM - Chịu ảnh hưởng nặng nề bởi tác động củ...,Theo báo cáo thị trường tháng 11.2021 của DKRA...,"Thứ năm, 09/12/2021 14:29 (GMT+7)",B. Chương,Bất động sản,"['Mua nhà đất', 'Codolet']"


In [4]:
drop_columns=['Date','Author(s)','Tags','URL']
df_news = df_news.drop(columns=drop_columns)
df_news.head(10)

,Title,Summary,Contents,Category
0,Hà Nội: Hô biến “đất vàng” thành chung cư cao ...,Sau khi di dời các cơ sở công nghiệp ra khỏi n...,"Sau cổ phần hóa, “đất vàng” thành chung cư Khả...",Bất động sản
1,Hà Nội và TP.HCM đối mặt với suy giảm nguồn cu...,Nguồn cung căn hộ giảm trên tất cả các phân kh...,Tại Báo cáo Kinh tế vĩ mô Quý I 2019 công bố m...,Bất động sản
2,"""Ăn theo"" Tp. Thủ Đức và sân bay Long Thành: S...",Cơn sốt đất chạy theo thông tin thành lập Thàn...,Giá đất tăng chóng mặt Có thể nói khắp các co...,Bất động sản
3,Mua nhà ở hình thành trong tương lai và những ...,Những kiến thức cơ bản về mua nhà ở hình thành...,Nhà ở hình thành trong tương lai là gì? Nhà ở ...,Bất động sản
4,"Condotel ""ế ẩm"", nhà đầu tư khủng hoảng",TPHCM - Chịu ảnh hưởng nặng nề bởi tác động củ...,Theo báo cáo thị trường tháng 11.2021 của DKRA...,Bất động sản
5,TMS Grand City Phúc Yên – “Dự án đáng sống năm...,"Ngày 9.7.2020, Tập đoàn TMS đã vinh dự được ôn...",“Dự án đáng sống 2020” là chương trình được tổ...,Bất động sản
6,Khởi động dự án xây chợ Giang Đình trong quần ...,"Nhằm kịp thời hỗ trợ cho các hộ kinh doanh, ti...",Chợ Giang Đình thuộc dự án khôi phục bến Giang...,Bất động sản
7,Dân bức xúc vì Phó Chủ tịch xã lấn chiếm 1000m...,"Ngày 20.10, UBND tỉnh Bình Định cho biết Chủ t...",Kết luận thanh tra đã chỉ ra hàng loạt sai phạ...,Bất động sản
8,Cư dân Vinhomes Hà Tĩnh hào hứng với đêm hội “...,"""Vui Trăng cùng Vinhomes"", vừa được tổ chức Ch...","Đến tham dự từ rất sớm, con trai của chị Lưu T...",Bất động sản
9,Thủ tục để xây nhà trên đất trồng cây lâu năm,Để có thể xây nhà ở trên đất trồng cây lâu năm...,Đất trồng cây lâu năm là gì? Theo Khoản 1 Đi...,Bất động sản


In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_news['label_id'] = label_encoder.fit_transform(df_news['Category'])

In [6]:
df_news.head(10)

,Title,Summary,Contents,Category,label_id
0,Hà Nội: Hô biến “đất vàng” thành chung cư cao ...,Sau khi di dời các cơ sở công nghiệp ra khỏi n...,"Sau cổ phần hóa, “đất vàng” thành chung cư Khả...",Bất động sản,1
1,Hà Nội và TP.HCM đối mặt với suy giảm nguồn cu...,Nguồn cung căn hộ giảm trên tất cả các phân kh...,Tại Báo cáo Kinh tế vĩ mô Quý I 2019 công bố m...,Bất động sản,1
2,"""Ăn theo"" Tp. Thủ Đức và sân bay Long Thành: S...",Cơn sốt đất chạy theo thông tin thành lập Thàn...,Giá đất tăng chóng mặt Có thể nói khắp các co...,Bất động sản,1
3,Mua nhà ở hình thành trong tương lai và những ...,Những kiến thức cơ bản về mua nhà ở hình thành...,Nhà ở hình thành trong tương lai là gì? Nhà ở ...,Bất động sản,1
4,"Condotel ""ế ẩm"", nhà đầu tư khủng hoảng",TPHCM - Chịu ảnh hưởng nặng nề bởi tác động củ...,Theo báo cáo thị trường tháng 11.2021 của DKRA...,Bất động sản,1
5,TMS Grand City Phúc Yên – “Dự án đáng sống năm...,"Ngày 9.7.2020, Tập đoàn TMS đã vinh dự được ôn...",“Dự án đáng sống 2020” là chương trình được tổ...,Bất động sản,1
6,Khởi động dự án xây chợ Giang Đình trong quần ...,"Nhằm kịp thời hỗ trợ cho các hộ kinh doanh, ti...",Chợ Giang Đình thuộc dự án khôi phục bến Giang...,Bất động sản,1
7,Dân bức xúc vì Phó Chủ tịch xã lấn chiếm 1000m...,"Ngày 20.10, UBND tỉnh Bình Định cho biết Chủ t...",Kết luận thanh tra đã chỉ ra hàng loạt sai phạ...,Bất động sản,1
8,Cư dân Vinhomes Hà Tĩnh hào hứng với đêm hội “...,"""Vui Trăng cùng Vinhomes"", vừa được tổ chức Ch...","Đến tham dự từ rất sớm, con trai của chị Lưu T...",Bất động sản,1
9,Thủ tục để xây nhà trên đất trồng cây lâu năm,Để có thể xây nhà ở trên đất trồng cây lâu năm...,Đất trồng cây lâu năm là gì? Theo Khoản 1 Đi...,Bất động sản,1


In [7]:
df_news[['Title', 'Contents','Category']].to_csv('data_news.csv',index=False)

In [8]:
df_news

,Title,Summary,Contents,Category,label_id
0,Hà Nội: Hô biến “đất vàng” thành chung cư cao ...,Sau khi di dời các cơ sở công nghiệp ra khỏi n...,"Sau cổ phần hóa, “đất vàng” thành chung cư Khả...",Bất động sản,1
1,Hà Nội và TP.HCM đối mặt với suy giảm nguồn cu...,Nguồn cung căn hộ giảm trên tất cả các phân kh...,Tại Báo cáo Kinh tế vĩ mô Quý I 2019 công bố m...,Bất động sản,1
2,"""Ăn theo"" Tp. Thủ Đức và sân bay Long Thành: S...",Cơn sốt đất chạy theo thông tin thành lập Thàn...,Giá đất tăng chóng mặt Có thể nói khắp các co...,Bất động sản,1
3,Mua nhà ở hình thành trong tương lai và những ...,Những kiến thức cơ bản về mua nhà ở hình thành...,Nhà ở hình thành trong tương lai là gì? Nhà ở ...,Bất động sản,1
4,"Condotel ""ế ẩm"", nhà đầu tư khủng hoảng",TPHCM - Chịu ảnh hưởng nặng nề bởi tác động củ...,Theo báo cáo thị trường tháng 11.2021 của DKRA...,Bất động sản,1
...,...,...,...,...,...
455,LD18121: Khốn khổ tột cùng người mẹ tật nguyền...,Chị Nguyễn Thị Hằng (44 tuổi) ở khu vực Thạnh ...,"Sinh ra trong một gia đình đông anh em, chị Hằ...",Tin hoạt động,17
456,LD17117: Hoàn cảnh trớ trêu của một gia đình c...,Cả 2 đứa con sinh ra mang trong mình những căn...,Đó là hoàn cảnh của gia đình bà Trịnh Thị Tuyế...,Tin hoạt động,17
457,LD17132: Khóc nghẹn cảnh cụ bà 80 tuổi nuôi 2 ...,"Mẹ mất từ 3 năm trước vì ung thư, nay cha lại ...","Vài ngày sau cái chết của anh Hiểu, khi chúng ...",Tin hoạt động,17
458,Prudential VN-Ủy ban ATGT Quốc gia chung tay c...,"Sáng nay (3.12), Ủy ban An toàn giao thông Quố...",Đây là một trong những hoạt động nhằm tạo cầu ...,Tin hoạt động,17


## NLP

In [9]:
# Một số hàm tiền xử lý văn bản cần thiết
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

stopword = set()
file = r'E:\NLP\Vietnamese-Text-Classification-master\data\vietnamese-stopwords.txt'
for line in open(file, "r",encoding="utf8"):
  line = line.strip('\n')
  stopword.add(line)

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

def text_preprocess(document):
    # chuẩn hóa unicode
    document = convert_unicode(document)
    # chuẩn hóa cách gõ dấu tiếng Việt
    document = chuan_hoa_dau_cau_tieng_viet(document)
    # tách từ
    document = word_tokenize(document, format="text")
    # xóa stopwords
    document = remove_stopwords(document)
    # đưa về lower
    document = document.lower()
    # xóa các ký tự không cần thiết
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)
    # xóa khoảng trắng thừa
    document = re.sub(r'\s+', ' ', document).strip()
    return document

In [10]:
df_news['Title'] = df_news['Title'].apply(lambda x : text_preprocess(x))
df_news['Contents'] = df_news['Contents'].apply(lambda x : text_preprocess(x))
df_news['text'] = df_news['Title'] + ' ' + df_news['Contents']

In [11]:
df_news['Category'].value_counts()

Bất động sản           20
Lưu trữ                20
Tin tức việc làm       20
Gia đình - Hôn nhân    20
Công nghệ              20
Tấm Lòng Vàng          20
Diễn đàn               20
Media                  20
Thể thao               20
Sức khỏe               20
Giáo dục               20
Lao Động & Đời sống    20
Xã hội                 20
Văn hóa - Giải trí     20
Thế giới               20
Xe +                   20
Pháp luật              20
Bạn đọc                20
Công đoàn              20
Thời sự                20
Kinh doanh             20
Lao Động cuối tuần     20
Tin hoạt động          20
Name: Category, dtype: int64

In [13]:
df_news = df_news.reset_index(drop=True)
df_news

,Title,Summary,Contents,Category,label_id,text
0,hà_nội hô_biến đất vàng thành chung_cư cao_tầng,Sau khi di dời các cơ sở công nghiệp ra khỏi n...,cổ_phần hóa đất vàng thành chung_cư khảo_sát l...,Bất động sản,1,hà_nội hô_biến đất vàng thành chung_cư cao_tần...
1,hà_nội tp hcm đối_mặt suy_giảm cung căn_hộ,Nguồn cung căn hộ giảm trên tất cả các phân kh...,báo_cáo kinh_tế vĩ_mô quý i 2019 công_bố mới_đ...,Bất động sản,1,hà_nội tp hcm đối_mặt suy_giảm cung căn_hộ báo...
2,ăn_theo tp thủ_đức sân_bay long thành sốt đất,Cơn sốt đất chạy theo thông tin thành lập Thàn...,giá đất chóng_mặt có_thể khắp đường quán cà_ph...,Bất động sản,1,ăn_theo tp thủ_đức sân_bay long thành sốt đất ...
3,mua hình_thành tương_lai,Những kiến thức cơ bản về mua nhà ở hình thành...,hình_thành tương_lai hình_thành tương_lai tài_...,Bất động sản,1,mua hình_thành tương_lai hình_thành tương_lai ...
4,condotel ế_ẩm nhà_đầu_tư khủng_hoảng,TPHCM - Chịu ảnh hưởng nặng nề bởi tác động củ...,báo_cáo thị_trường 11 2021 dkra vietnam loại_h...,Bất động sản,1,condotel ế_ẩm nhà_đầu_tư khủng_hoảng báo_cáo t...
...,...,...,...,...,...,...
455,ld18121 khốn_khổ tột_cùng mẹ tật_nguyền nuôi c...,Chị Nguyễn Thị Hằng (44 tuổi) ở khu vực Thạnh ...,sinh gia_đình đông anh_em hằng bươn_chải làm_t...,Tin hoạt động,17,ld18121 khốn_khổ tột_cùng mẹ tật_nguyền nuôi c...
456,ld17117 hoàn_cảnh trớ_trêu gia_đình 2 tật_nguyền,Cả 2 đứa con sinh ra mang trong mình những căn...,hoàn_cảnh gia_đình trịnh_thị_tuyết sn 1969 lê ...,Tin hoạt động,17,ld17117 hoàn_cảnh trớ_trêu gia_đình 2 tật_nguy...
457,ld17132 khóc nghẹn_cảnh cụ 80 nuôi 2 cháu mồ_côi,"Mẹ mất từ 3 năm trước vì ung thư, nay cha lại ...",chết chúng_tôi nghi_ngút khói_hương vương_thị_...,Tin hoạt động,17,ld17132 khóc nghẹn_cảnh cụ 80 nuôi 2 cháu mồ_c...
458,prudential vn ủy ban_atgt quốc_gia chia_sẻ nỗi...,"Sáng nay (3.12), Ủy ban An toàn giao thông Quố...",hoạt_động cầu_nối nhân_ái cộng_đồng nạn_nhân g...,Tin hoạt động,17,prudential vn ủy ban_atgt quốc_gia chia_sẻ nỗi...


In [15]:
 # Thống kê các word xuất hiện ở tất cả các nhãn
total_label = 9
# Lấy túi từ vựng của dữ liệu đầu vào
vocab = {}
label_vocab = {}
for idx, row in  df_news.iterrows():
    words = row['text'].split()
    label = row['Category']
    if label not in label_vocab:
        label_vocab[label] = {}
    for word in words:
        label_vocab[label][word] = label_vocab[label].get(word, 0) + 1
        if word not in vocab:
            vocab[word] = set()
        vocab[word].add(label)

In [16]:
vocab

{'hà_nội': {'Bạn đọc',
  'Bất động sản',
  'Công đoàn',
  'Diễn đàn',
  'Gia đình - Hôn nhân',
  'Giáo dục',
  'Kinh doanh',
  'Lao Động & Đời sống',
  'Lao Động cuối tuần',
  'Lưu trữ',
  'Pháp luật',
  'Sức khỏe',
  'Thể thao',
  'Thời sự',
  'Tin hoạt động',
  'Tin tức việc làm',
  'Tấm Lòng Vàng',
  'Văn hóa - Giải trí',
  'Xe +',
  'Xã hội'},
 'hô_biến': {'Bất động sản', 'Lao Động cuối tuần'},
 'đất': {'Bạn đọc',
  'Bất động sản',
  'Gia đình - Hôn nhân',
  'Kinh doanh',
  'Lao Động & Đời sống',
  'Lao Động cuối tuần',
  'Pháp luật',
  'Thế giới',
  'Thể thao',
  'Thời sự',
  'Tin hoạt động',
  'Tin tức việc làm',
  'Tấm Lòng Vàng',
  'Xã hội'},
 'vàng': {'Bạn đọc',
  'Bất động sản',
  'Diễn đàn',
  'Gia đình - Hôn nhân',
  'Giáo dục',
  'Kinh doanh',
  'Lao Động & Đời sống',
  'Lao Động cuối tuần',
  'Lưu trữ',
  'Media',
  'Thể thao',
  'Thời sự',
  'Tin hoạt động',
  'Tấm Lòng Vàng',
  'Văn hóa - Giải trí',
  'Xe +',
  'Xã hội'},
 'thành': {'Bạn đọc',
  'Bất động sản',
  'Công 

In [63]:
# Chia tập train/test
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import np_utils
from keras.utils import np_utils
test_percent = 0.2

In [66]:
X_train, X_test, y_train, y_test = train_test_split(df_news['text'], df_news['Category'], test_size=test_percent, random_state=42)
unique_classes_train = np.unique(y_train)
unique_classes_test = np.unique(y_test)

print("Unique classes in Y_train:", unique_classes_train)
print("Unique classes in Y_test:", unique_classes_test)
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
label_encoder.fit(y_test)
print(list(label_encoder.classes_), '\n')
Y_train = label_encoder.transform(y_train)
Y_test = label_encoder.transform(y_test)
num_classes = len(np.unique(Y_train))  # Calculate the actual number of classes
Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test = np_utils.to_categorical(Y_test, num_classes)

Unique classes in Y_train: ['Bạn đọc' 'Bất động sản' 'Công nghệ' 'Công đoàn' 'Diễn đàn'
 'Gia đình - Hôn nhân' 'Giáo dục' 'Kinh doanh' 'Lao Động & Đời sống'
 'Lao Động cuối tuần' 'Lưu trữ' 'Media' 'Pháp luật' 'Sức khỏe' 'Thế giới'
 'Thể thao' 'Thời sự' 'Tin hoạt động' 'Tin tức việc làm' 'Tấm Lòng Vàng'
 'Văn hóa - Giải trí' 'Xe +' 'Xã hội']
Unique classes in Y_test: ['Bạn đọc' 'Bất động sản' 'Công nghệ' 'Công đoàn' 'Diễn đàn'
 'Gia đình - Hôn nhân' 'Giáo dục' 'Kinh doanh' 'Lao Động & Đời sống'
 'Lao Động cuối tuần' 'Lưu trữ' 'Media' 'Pháp luật' 'Sức khỏe' 'Thế giới'
 'Thể thao' 'Thời sự' 'Tin hoạt động' 'Tin tức việc làm' 'Tấm Lòng Vàng'
 'Văn hóa - Giải trí' 'Xe +' 'Xã hội']
['Bạn đọc', 'Bất động sản', 'Công nghệ', 'Công đoàn', 'Diễn đàn', 'Gia đình - Hôn nhân', 'Giáo dục', 'Kinh doanh', 'Lao Động & Đời sống', 'Lao Động cuối tuần', 'Lưu trữ', 'Media', 'Pháp luật', 'Sức khỏe', 'Thế giới', 'Thể thao', 'Thời sự', 'Tin hoạt động', 'Tin tức việc làm', 'Tấm Lòng Vàng', 'Văn hóa - Giải trí',

In [67]:
dict_label = {}
for idx, val in enumerate(list(label_encoder.classes_)):
  dict_label[str(idx)] = val
pickle.dump(dict_label,open('dict_label.pkl', 'wb'))

In [68]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

max_words = len(vocab.keys())

tok = Tokenizer(num_words = max_words)
tok.fit_on_texts(X_train)
pickle.dump(tok,open('tokenize_text.pkl', 'wb'))

sequences = tok.texts_to_sequences(X_train)
max_len = 1600

sequences_matrix = pad_sequences(sequences,maxlen = max_len,padding='post')

print('Câu thứ nhất:', X_train[1])
print('Label câu thứ nhất:', y_train[1])
print('Độ dài: ', len(sequences[1]))
print('Tổng số câu', len(sequences))
print('Sequences_matrix shape: ', sequences_matrix.shape)
print('Sequences_matrix câu thứ nhất: ', sequences_matrix[1, :])

Câu thứ nhất: hà_nội tp hcm đối_mặt suy_giảm cung căn_hộ báo_cáo kinh_tế vĩ_mô quý i 2019 công_bố mới_đây viện nghiên_cứu kinh_tế chính_sách đánh_giá thị_trường bất_động_sản thành_phố hỗ_trợ cầu tuy_nhiên thị_trường căn_hộ đối_mặt suy_giảm cung căn_hộ hai thành_phố hà_nội tp hcm báo_cáo hà_nội căn_hộ hạng a 84 hạng b c 35 đầu quý kỳ 2018 số_lượng căn_hộ tung thị_trường 76 căn_hộ tầm_trung phân_khúc ưa_chuộng chiếm 70 số_lượng giao_dịch savills hà_nội giá căn_hộ hạng b 1 390 usdm2 2 quý 8 tp hcm diễn_biến xảy tương_tự số_lượng căn_hộ tung thị_trường 4500 51 5 quý 57 quý 12018 giải_thích nguyên_nhân cung căn_hộ pgs ts nguyễn_đức_thành viện trưởng vepr quá_trình thẩm_duyệt pháp_lý kéo_dài ngoài_ra thị_trường căn_hộ hạng c thu_nhập chiếm 85 thị_phần tập_trung ngoại_thành đối_diện cung nhu_cầu dự_báo cung quý tiếp_theo cải_thiện nhanh_chóng 34 dự_án gia_nhập thị_trường vincity ocean park brg smart city hà_nội quá_trình xét_duyệt pháp_lý đẩy tuy_nhiên ngân_hàng liên_tục siết vốn vay lãi_suất

In [72]:
from keras.models import Sequential
from keras import layers
from keras.callbacks import EarlyStopping

embedding_dim =1000
num_filters = 128
kernel_size = 7
vocab_size = 49000

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size,
                           output_dim=embedding_dim, 
                           input_length=1600))
model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
model.add(layers.Conv1D(64, kernel_size, activation='relu'))
model.add(layers.Conv1D(28, kernel_size, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(18, activation='relu'))
model.add(layers.Dense(21, activation='softmax'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(sequences_matrix, Y_train, validation_split = 0.1,
batch_size=32, epochs=10, verbose=1)

ModuleNotFoundError: No module named 'keras.src.engine.base_layer_v1'

In [80]:
test_sequences = tok.texts_to_sequences(X_test)
max_len = 1600

test_sequences_matrix = pad_sequences(test_sequences,maxlen = max_len,padding='post')
score = model.evaluate(test_sequences_matrix, Y_test, verbose=0)
print(score)

y_predict = model.predict(test_sequences_matrix)

[0.19267897307872772, 0.8249551057815552]
35/35 [==============================] - 15s 415ms/step


In [81]:
i = 0
print(f"Câu test: {list(X_test)[i]}\nGiá trị dự đoán: {dict_label[str(np.argmax(y_predict[0]))]}")

Câu test: tết đồ_thừa tết 20 mẹ chồng khánh vân bỉm_sơn thanh_hóa gọi điện_thông_báo đụng lợn 80 kg ngăn mẹ nàng dâu dặn tùy tình_hình cân_đối hai bận bốn người_lớn hai đứa tết mẹ chốt đụng lợn vợ_chồng gửi 5 triệu đồng sắm tết vân _bận kinh_doanh trưa 30 tết vợ_chồng bố_mẹ lo tươm_tất kết_hôn vân phát hoảng thực_phẩm đồ_ăn khắp nia bếp bày hai chục bánh_chưng tủ_lạnh chất 5 giò_lụa bếp nồi thịt đông riu_riu lửa nồi cá kho xong sân giếng treo giò_xào hai xâu nem_chua treo lủng_lẳng mẹ mua lá nem khéo lắm sợ rách bảo tủ_lạnh rán thả rán sơ mẹ vân 50 nem _lượng thực_phẩm đồ nấu xong thắp hương vân phụ mẹ chuẩn_bị cơm_cúng tàn_hương gia_đình hạ mâm đồ_ăn nguội_ngắt hai bữa đầu đồ ngon chán thành_thử món đạm bánh_chưng cúng ế bữa bữa mâm cơm_mới hộp túi nồi đựng đồ ăn_thừa ngày_một tiêu_thụ phần_nào thậm_chí đồ ăn_thừa vân đi tiếc bố_mẹ đi thậm_chí đổ_lộn giò thịt_luộc thịt đông nồi bữa đun mùng 3 tết bụng_dạ hầu_như đình_công bố_mẹ uống men tiêu_hóa mẹ đi ruộng bẻ ngô luộc nhẹ_bụng vợ_chồ

In [58]:
model.save("model_cnn_news.h5")